# Production Code

In [ ]:
import os
import re
import numpy as np
import pandas as pd

source_directory = "../surveys/plain_txt/"

## Extract

### Global Variables

In [ ]:
species_list = ["toro para carne \/ ?\n{0,2}novillo \(\+300 kg\)\n", "vaca\n", "ternero\n", "vaquilla\n", 
                "bovino macho ?\n{0,2}reproductor\n", "llama macho para carne\n", "lama hembra\n",
                "llama macho reproductor\n", "borrego \(macho joven\)\n", "cordero\n", "capón\n",
                "oveja\n", "ovino macho adulto ?\n\npara carne\n", 
                "[^bB]ovino macho reproductor\n", "chivo \(macho adulto\)\n", 
                "caprino reproductor macho ?\n\nadulto \(castrón\)\n",
                "cabra \(hembra\)\n", "chivo capón \(castrado\)\n", "cabrito \/ chivito mamón\n",
                "cabrito \/ chivito lechón\n", "burro macho\n", "burro hembra\n",
                "equino macho reproductor\n", "equino macho para carne\n",
                "equino hembra \/ yegua\n", "potrillo \/ potranca\n", 
                "equino manso \(macho \/ hembra\)\n", "mular\n", 
                "cerdo macho reproductor \n*\(padrillo\)\n", "cerda \(madre\) hembra\n", 
                "maltón \(cachorro\)\n", "cerdo capón \(castrado\) ?\/ ?cerda\n* ?para\n* *carne\n",
                "lechón \/ lechona\n", "cerdo macho adulto para carne\n", "v-2:"]
translation_list = ["bull_or_steer_for_meat_300_+_kg", "cow", "calf",
                    "heifer", "male_bovine_breeding",
                    "male_llama_for_meat", "female_llama",
                    "male_llama_breeding", "male_sheep_young", "lamb",
                    "capon", "female_sheep", "male_sheep_adult_for_meat",
                    "male_sheep_breeding", "male_goat", "male_goat_breeding",
                    "female_goat", "male_goat_castrated", "goat_young_suckling_mamón",
                    "goat_young_suckling_lechón", "male_donkey", "female_donkey",
                    "male_equine_breeding", "male_equine_for_meat",
                    "female_equine", "equine_young", "equine_tame",
                    "mule", "male_pig_breeding", "female_pig_breeding",
                    "young_animal", "male_pig_castrated_or_for_meat", "pig_suckling",
                    "male_pig_for_meat"]
# column headers not including species column
column_headers = ["has_sold", "month_of_sale_1", "number_1",
                  "unit_price_1", "month_of_sale_2", "number_2",
                  "unit_price_2", "month_of_sale_3", "number_3",
                  "unit_price_3", "market_destination"]

### Helper Functions

In [ ]:
def find_between(str_1, str_2, contents):
    regex_between = str_1 + r"([\w\W]*)" + str_2
    regex_search = re.findall(regex_between, contents)

    return regex_search

### Parsing Functions

In [ ]:
def parse_interviewee(contents):
    name_regex = r"nombre de la persona[^\n:]*:[\s]*([^\n]*)\n"
    name_search = re.findall(name_regex, contents)
    if len(name_search) == 1:
        name = name_search[0]
        if name == "3":
            name = "N/A"
    else:
        alt_name_regex = r"nombre de la persona[^\n\(]*\(\w* +([^\n]*)\n"
        alt_name_search = re.findall(alt_name_regex, contents)
        if len(alt_name_search) == 1:
            name = alt_name_search[0]
            if name == "3":
                name = "N/A"
        else:
            name = "UNENCOUNTERED FORMAT"

    name = name.strip()

    return name


def parse_date_range(section_search):
    date_range = ""

    if len(section_search) == 1:
        section_string = section_search[0]

        date_regex = r"([\w ]+\d+ *[-–] *[\w ]+\d+)"
        date_search = re.findall(date_regex, section_string)

        if len(date_search) == 1:
            date_range = date_search[0]
        else:
            date_range = "UNENCOUNTERED FORMAT"
    else:
        date_range = "UNENCOUNTERED FORMAT"

    return date_range


def parse_section_5_1(section_search, species_list,
                      translation_list, column_headers):
    section_5_1_data = []

    if len(section_search) == 1:
        section_string = section_search[0] + "v-2:"

        for i in range(len(species_list)-1):
            item_dict = {}
            item_1 = species_list[i]
            item_2 = species_list[i+1]
            term = translation_list[i]

            row_search = find_between(item_1, item_2, section_string)

            item_dict["species"] = term

            # if species row found
            if len(row_search) == 1:
                row_string = row_search[0]
                row_list = row_string.split("\n")
                row_list_len = len(row_list)

                if row_list_len >= 23:
                    # skip odd indexes (1, 3, 5, 7)
                    # drop first two rows (start count at -3)
                    count = -1
                    if (row_list[-1 * count] == ""
                       and row_list[-1 * count + 2] != ""):
                        count -= 2

                    for j in range(row_list_len):
                        if count > 20:
                            break
                        if count >= 0 and count % 2 == 0:
                            col = column_headers[count//2]
                            entry = row_list[j]
                            entry = entry.strip()
                            item_dict[col] = entry
                        count += 1

                else:
                    for col in column_headers:
                        item_dict[col] = "UNENCOUNTERED FORMAT"
            else:
                for col in column_headers:
                    item_dict[col] = "UNENCOUNTERED FORMAT"

            section_5_1_data.append(item_dict)
    else:
        section_5_1_data = [{}]

    return section_5_1_data

### Process the Whole File

In [ ]:
def process_files(source_directory, species_list,
                  translation_list, column_headers):
    dataset = []

    for process_file in os.listdir(source_directory):
        file_path = os.path.join(source_directory, process_file)

        # with statements automatically control the closing of files
        with open(file_path, "r") as file:
            contents = file.read()
            contents = contents.lower()

            data_dict = {}

            section_start = "v-1: ventas de ganado"
            section_end = "v-2: ventas de"
            # extract only the relevant section
            section_search = find_between(section_start, section_end, contents)

            interviewee = parse_interviewee(contents)
            date_range = parse_date_range(section_search)
            section_5_1_data = parse_section_5_1(section_search, species_list,
                                                 translation_list, column_headers)

            # check to make sure files contain the right amount of entries
            if len(section_5_1_data) <= 1:
                print(f"Unencountered Format: {process_file}\n"
                      + f"Length of data: {len(section_5_1_data)}\n")

            for row in section_5_1_data:
                data_dict = {}
                data_dict["filename"] = process_file
                data_dict["interviewee"] = interviewee
                data_dict["date_range"] = date_range

                for key, value in row.items():
                    data_dict[key] = value

                dataset.append(data_dict)

    # convert list to DataFrame
    raw_df = pd.DataFrame(data=dataset)

    return raw_df

## Transform

### Helper Functions

In [ ]:
def clean_date_range(dataf):
    replace_dict = {"agosto 2018 - agosto 2019": "august 2018-2019",
                    "septiembre 2018 - septiembre 2019": "september 2018-2019",
                    "octubre 2018 – octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre 2019": "october 2018-2019",
                    "octubre 2018 - octubre2019": "october 2018-2019"}
    dataf["date_range"] = dataf["date_range"].replace(replace_dict)

    return dataf


def clean_has_sold(dataf):
    map_dict = {"si": "yes",
                "s1": "yes",
                "bo": "no"}

    dataf["has_sold"] = dataf["has_sold"].replace(map_dict)

    return dataf


def clean_month_of_sale(dataf):
    map_dict = {"01": "1",
                "02": "2",
                "03": "3",
                "04": "4",
                "05": "5",
                "06": "6",
                "07": "7",
                "08": "8",
                "09": "9",
                "10": "10",
                "11": "11",
                "12": "12",
                "junio": "6",
                "jun": "6",
                "mayo": "5",
                "mar": "3",
                "julio": "7",
                "dic": "12",
                "diciembre": "12",
                "abril": "4",
                "marzo": "3",
                "ene": "1",
                "junio, julio": "6,7",
                "enero": "1",
                "noviembre": "11",
                "octubre": "10",
                "abril, mayo": "4,5",
                "abr": "4",
                "abri": "4",
                "mayo, junio": "5,6",
                "marz": "3",
                "juni": "6",
                "1 a 12": "12,1",
                "sept": "9",
                "nov": "11",
                "may": "5",
                "agos": "8",
                "oct": "10",
                "ener": "1",
                "juni0": "6",
                "agosto, septiembre": "8,9",
                "septiembre": "9",
                "agosto": "8",
                "dici.": "12",
                "noviem,bre": "11",
                "junio, julio, agosto,septiembre": "6,7,8,9",
                "ene.": "1",
                "0": ""}

    dataf["month_of_sale_1"] = dataf["month_of_sale_1"].replace(map_dict)
    dataf["month_of_sale_2"] = dataf["month_of_sale_2"].replace(map_dict)
    dataf["month_of_sale_3"] = dataf["month_of_sale_3"].replace(map_dict)

    return dataf


def clean_unit_price(dataf):
    replace_filter_1 = dataf["unit_price_1"].str.contains(r"\.\d{3}", na=False)
    replace_filter_2 = dataf["unit_price_2"].str.contains(r"\.\d{3}", na=False)
    replace_filter_3 = dataf["unit_price_3"].str.contains(r"\.\d{3}", na=False)

    replace_series_1 = dataf["unit_price_1"].str.replace(".", "")
    replace_series_2 = dataf["unit_price_2"].str.replace(".", "")
    replace_series_3 = dataf["unit_price_3"].str.replace(".", "")

    dataf["unit_price_1"] = dataf["unit_price_1"].where(~replace_filter_1,
                                                        replace_series_1)
    dataf["unit_price_2"] = dataf["unit_price_2"].where(~replace_filter_2,
                                                        replace_series_2)
    dataf["unit_price_3"] = dataf["unit_price_3"].where(~replace_filter_3,
                                                        replace_series_3)

    dataf["unit_price_1"] = dataf["unit_price_1"].str.replace("$", "")
    dataf["unit_price_2"] = dataf["unit_price_2"].str.replace("$", "")
    dataf["unit_price_3"] = dataf["unit_price_3"].str.replace("$", "")

    replace_dict = {"venta en casa del productor": ""}
    dataf["unit_price_3"] = dataf["unit_price_3"].replace(replace_dict)

    return dataf


def clean_market_destination(dataf):
    replace_dict = {"1-2": "1,2",
                    "1-3": "1,3",
                    "1 - 3": "1,3"}
    dataf["market_destination"] = dataf["market_destination"].replace(replace_dict)

    return dataf

### Pipeline Functions

In [ ]:
def start_pipeline(dataf):
    return dataf.copy()


def handle_null_data(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = ((isnull_sum >= 11)
                   | ((isnull_sum == 10) & (dataf["has_sold"] == "no")))
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf


def clean_values(dataf):
    dataf = clean_date_range(dataf)
    dataf = clean_unit_price(dataf)
    dataf = clean_market_destination(dataf)
    dataf = clean_has_sold(dataf)
    dataf = clean_month_of_sale(dataf)

    return dataf

### Create Raw and Cleaned DataFrame

In [ ]:
raw_df = process_files(source_directory, species_list,
                       translation_list, column_headers)

clean_df = (raw_df
            .pipe(start_pipeline)
            .pipe(clean_values)
            .pipe(handle_null_data))

## Other
### Functions to Check the Implementation

In [ ]:
def get_dropped_df(raw_dataf, clean_dataf):
    raw_indexes = raw_dataf.index
    clean_indexes = clean_dataf.index

    dropped_indexes = raw_indexes[~raw_indexes.isin(clean_indexes)]
    dropped_df = raw_dataf.loc[dropped_indexes].copy()

    return dropped_df

### Create Dropped DataFrame
Create the `dropped_df` and collect the rows which contain no non-null values

In [ ]:
dropped_df = get_dropped_df(raw_df, clean_df)

# get rows that were dropped but do not contain a has_sold with specific values
null_df = dropped_df.loc[~dropped_df["has_sold"].isin(["no", "bo"]),
                         ["filename", "species"]]

### Check Dropped Row

In [ ]:
def get_dropped_row(section_search, species,
                    species_list, translation_list):
    dropped_row = {}

    if len(section_search) == 1:
        section_string = section_search[0] + "v-2:"

        dropped_row["species"] = species

        species_index = translation_list.index(species)
        item_1 = species_list[species_index]
        item_2 = species_list[species_index + 1]

        row_search = find_between(item_1, item_2, section_string)

        # if species row found
        if len(row_search) == 1:
            row_string = row_search[0]
            row_list = row_string.split("\n")
            row_list_len = len(row_list)

            if row_list_len >= 23:
                count = 0
                for j in range(row_list_len):
                    if count > 22:
                        break
                    if count >= 0:
                        col = str(count)
                        entry = row_list[j]
                        entry = entry.strip()
                        dropped_row[col] = entry
                    count += 1

            else:
                for j in range(23):
                    dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
        else:
            for j in range(23):
                dropped_row[str(j)] = "UNENCOUNTERED FORMAT"
    else:
        dropped_row = {}

    return dropped_row

### Check Dropped Rows for Parsing Errors

In [ ]:
def check_dropped_rows(source_directory, species_list, translation_list, s):
    process_file = s["filename"]
    species = s["species"]

    check_dict = {}

    file_path = os.path.join(source_directory, process_file)

    # with statements automatically control the closing of files
    with open(file_path, "r") as file:
        contents = file.read()
        contents = contents.lower()

        section_start = "v-1: ventas de ganado"
        section_end = "v-2: ventas de"
        section_search = find_between(section_start, section_end, contents)

        dropped_row = get_dropped_row(section_search, species,
                                      species_list, translation_list)

        # check to make sure files contain the right amount of entries
        if len(dropped_row) <= 1:
            print(f"Unencountered Format: {process_file}\n"
                  + f"Length of data: {len(dropped_row)}\n")

        check_dict["filename"] = process_file

        for key, value in dropped_row.items():
            check_dict[key] = value

    # convert list to Series
    check_series = pd.Series(data=check_dict)

    return check_series

In [ ]:
check_df = null_df.apply(lambda s: check_dropped_rows(source_directory,
                                                      species_list,
                                                      translation_list,
                                                      s),
                         axis=1)

In [ ]:
def drop_null_check(dataf):
    # fill missing values with null
    dataf = dataf.replace("", np.NaN)

    # drop rows which contain no information
    isnull_sum = dataf.isnull().sum(axis=1)
    drop_filter = isnull_sum >= 23
    drop_indexes = dataf[drop_filter].index

    dataf = dataf.drop(drop_indexes)

    return dataf

In [ ]:
non_null_df = (check_df
               .pipe(start_pipeline)
               .pipe(drop_null_check))

In [ ]:
non_null_df

## Load
### Export to CSV

In [ ]:
if non_null_df.shape[0] == 0:
    datasets_directory = "../datasets/"
    filename = "section_5_1.csv"
    file_path = os.path.join(datasets_directory, filename)

    clean_df.to_csv(file_path, index=False, na_rep="null")

    print(f"Exported to {filename}")
else:
    print("Error: Dropped rows contain non-null values")

***
# Testing Code
### View the Data

In [ ]:
clean_df.head(50)

### Check for Parsing Errors

In [ ]:
cols_to_check = clean_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(clean_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Check Dropped Rows
Another way to check is by opening the variable inspector and manually scrolling
through the `df_dropped` DataFrame. 

The code below tells us whether or not there are files that contain no information for this section. If raw file count is the same as clean file count, then no files were dropped.

In [ ]:
print("Raw File Count: {}".format(len(raw_df["filename"].value_counts())))
print("Clean File Count: {}".format(len(clean_df["filename"].value_counts())))

If there are files that are completely dropped, find those files and look at them to check for parsing errors.

In [ ]:
raw_file_names = raw_df["filename"].value_counts().index
clean_file_names = clean_df["filename"].value_counts().index

raw_file_names[~raw_file_names.isin(clean_file_names)]

Check the rows value counts to see if any rows are being dropped that should not be.

In [ ]:
cols_to_check = dropped_df.columns
cols_to_drop = ["filename", "interviewee"]

cols_to_check = cols_to_check.drop(cols_to_drop)

for col in cols_to_check:
    print("*" * 50)
    print(" " * 5 + col)
    print(dropped_df[col].value_counts(dropna=False))
    print("*" * 50)
    print("\n")

### Duplicate Rows

In [ ]:
df_columns = clean_df.columns[clean_df.columns != "filename"]
clean_df[clean_df.loc[:, df_columns].duplicated(keep=False)]